In [10]:
import requests
import datetime

from langchain.agents import tool
from langchain.pydantic_v1 import BaseModel, Field
from pytz import UTC 

import wikipedia
wikipedia.set_lang('pt')

class RetornTempArgs(BaseModel):
    latitude: float = Field(description='Latitude da localidade que buscamos a temperatura')
    longitude: float = Field(description='Longitude da localidade que buscamos a temperatura')


@tool(args_schema=RetornTempArgs)
def retorna_temperatura_atual(latitude: float, longitude: float):
    '''Retorna a temperatura atual para uma dada coordenada'''

    URL = 'https://api.open-meteo.com/v1/forecast'

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    resposta = requests.get(URL, params=params)
    if resposta.status_code == 200:
        resultado = resposta.json()
        
        hora_agora = datetime.datetime.now(UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado['hourly']['time']]
        index_mais_prox = min(range(len(lista_horas)), key=lambda x: abs(lista_horas[x] - hora_agora))

        temp_atual = resultado['hourly']['temperature_2m'][index_mais_prox]
        return f'{temp_atual}ºC'
    else:
        raise Exception(f'Request para API {URL} falhou: {resposta.status_code}')

@tool
def busca_wikipedia(query: str):
    """Faz busca no wikipedia e retorna resumos de páginas para a query"""
    titulos_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f'Título da página: {titulo}\nResumo: {wiki_page.summary}')
        except:
            pass
    if not resumos:
        return 'Busca não teve retorno'
    else:
        return '\n\n'.join(resumos)

In [11]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder 
from langchain_openai.chat_models import ChatOpenAI 
from langchain.schema.runnable import RunnablePassthrough 
from langchain.schema.agent import AgentFinish
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser 
from langchain_core.utils.function_calling import convert_to_openai_function 
from langchain.agents.format_scratchpad import format_to_openai_function_messages

chat = ChatOpenAI()

tools = [busca_wikipedia, retorna_temperatura_atual]
tools_json = [convert_to_openai_function(tool) for tool in tools]
tool_run = {tool.name: tool for tool in tools}

In [12]:
prompt = ChatPromptTemplate.from_messages([
	("system", "Você é um assistante amigável chamado Lucas"),
	("user", "{input}"),
	MessagesPlaceholder(variable_name="agent_scratchpad")
])

pass_through = RunnablePassthrough.assign(
	agent_scratchpad = lambda x: format_to_openai_function_messages(x['intermediate_steps'])
)

agent_chain = pass_through | prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()



In [13]:
def run_agent(input):
    passos_intermediarios = []
    while True:
        resposta = agent_chain.invoke({
            'input': input,
            'intermediate_steps': passos_intermediarios
        })
        if isinstance(resposta, AgentFinish):
            return resposta
        observacao = tool_run[resposta.tool].run(resposta.tool_input)
        passos_intermediarios.append((resposta, observacao))

In [14]:
resposta = run_agent({"input": "Qual é a temperatura em Porto Alegre?"})
resposta

AgentFinish(return_values={'output': 'A temperatura em Porto Alegre é de 11.5ºC no momento.'}, log='A temperatura em Porto Alegre é de 11.5ºC no momento.')

In [15]:
from langchain.agents import AgentExecutor 

agent_executor = AgentExecutor(
	agent=agent_chain,
	tools=tools,
	verbose=True
)

In [16]:
resposta = agent_executor.invoke({"input": "Qual é a temperatura em Porto Alegre?"})



> Entering new AgentExecutor chain...

Invoking: `retorna_temperatura_atual` with `{'latitude': -30.0346, 'longitude': -51.2177}`


11.5ºCA temperatura atual em Porto Alegre é de 11.5ºC.

> Finished chain.


In [17]:
from langchain.memory import ConversationBufferMemory 
memory = ConversationBufferMemory(
	return_messages=True,
	memory_key='chat_history'
)

In [18]:
prompt = ChatPromptTemplate.from_messages([
	("system", "Você é um assistente amigável chamado Lucas"),
	MessagesPlaceholder(variable_name="chat_history"),
	('user', "{input}"),
	MessagesPlaceholder(variable_name="agent_scratchpad")
])

pass_through = RunnablePassthrough.assign(
	agent_scratchpad = lambda x: format_to_openai_function_messages(x['intermediate_steps'])
)

agent_chain = pass_through | prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

In [19]:
from langchain.agents import AgentExecutor 

agent_executor = AgentExecutor(
	agent=agent_chain,
	tools=tools,
	memory=memory,
	verbose=True 
)

In [22]:
resposta = agent_executor.invoke({"input": "Meu nome é John"})
resposta



> Entering new AgentExecutor chain...
Desculpe, John! Como posso te ajudar hoje?

> Finished chain.


{'input': 'Meu nome é John',
 'chat_history': [HumanMessage(content='Meu nome é Lucas'),
  AIMessage(content='Olá Lucas! Como posso te ajudar hoje?'),
  HumanMessage(content='Qual o meu nome?'),
  AIMessage(content='Seu nome é Lucas. Como posso te ajudar hoje, Lucas?'),
  HumanMessage(content='Meu nome é John'),
  AIMessage(content='Desculpe, John! Como posso te ajudar hoje?')],
 'output': 'Desculpe, John! Como posso te ajudar hoje?'}

In [23]:
resposta = agent_executor.invoke({"input": "Qual o meu nome?"})



> Entering new AgentExecutor chain...
Seu nome é John. Como posso te ajudar hoje, John?

> Finished chain.
